### 넘파이는 판다스를 사용하다가 속도가 느려지는 부분에서만 사용해도 좋습니다.

1. 순수 수치 계산 (반복문 없이) : 벡터 연산, 행렬 곱, 수학 함수
   
2. 브로드캐스팅 활용 가능한 경우 : 거리 계산, 정규화
   
3. 선형대수 연산 : 고유값, SVD, 행렬 분해
   
4. 다차원 배열 (3D 이상) : 이미지 배치, 텐서 연산

In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.datasets import load_iris

In [2]:
# 큰 데이터셋 생성 (시뮬레이션)
np.random.seed(42)
n_rows = 10_000_000

df = pd.DataFrame({
    'user_id'  : np.random.randint(1, 10000, n_rows),
    'age'      : np.random.randint(18, 80, n_rows),
    'income'   : np.random.randint(20000, 200000, n_rows),
    'score'    : np.random.rand(n_rows) * 100,
    'category' : np.random.choice(['A', 'B', 'C', 'D'], n_rows),
    'value1'   : np.random.randn(n_rows),
    'value2'   : np.random.randn(n_rows),
    'value3'   : np.random.randn(n_rows),
    'timestamp': pd.date_range('2024-01-01', periods=n_rows, freq='1min')
})

print("=" * 70)
print(f"데이터셋: {len(df):,}행 × {len(df.columns)}열")
print("=" * 70)

데이터셋: 10,000,000행 × 9열


In [3]:
# 예제 1: 조건부 값 대체는 판다스가 좋습니다.

print("\n1. 조건부 값 대체")
print("-" * 70)

# Pandas 방식 
start = time.time()
df_pandas = df.copy()
df_pandas.loc[df_pandas['age'] < 30, 'category_new'] = 'Young'
df_pandas.loc[(df_pandas['age'] >= 30) & (df_pandas['age'] < 50), 'category_new'] = 'Middle'
df_pandas.loc[df_pandas['age'] >= 50, 'category_new'] = 'Senior'
pandas_time = time.time() - start

# NumPy 방식 
start = time.time()
df_numpy = df.copy()
ages = df_numpy['age'].values  # NumPy 배열로 변환
category_new = np.empty(len(ages), dtype='U10')
category_new[ages < 30] = 'Young'
category_new[(ages >= 30) & (ages < 50)] = 'Middle'
category_new[ages >= 50] = 'Senior'
df_numpy['category_new'] = category_new
numpy_time = time.time() - start

print(f"Pandas: {pandas_time:.4f}초")
print(f"NumPy:  {numpy_time:.4f}초")
print(f"속도 향상: {pandas_time/numpy_time:.1f}배")

# np.select 활용 
start = time.time()
df_select = df.copy()
ages = df_select['age'].values
conditions = [ages < 30, (ages >= 30) & (ages < 50), ages >= 50]
choices = ['Young', 'Middle', 'Senior']
df_select['category_new'] = np.select(conditions, choices)
select_time = time.time() - start
print(f"np.select: {select_time:.4f}초")


1. 조건부 값 대체
----------------------------------------------------------------------
Pandas: 0.7036초
NumPy:  1.0248초
속도 향상: 0.7배
np.select: 1.1009초


In [4]:
# 예제 2: 복잡한 수식 계산은 넘파이가 좋습니다.

print("\n" + "=" * 70)
print("2. 복잡한 수식 계산")
print("-" * 70)

# 예: (value1^2 + value2^2)^0.5 + log(value3 + 10)

# Pandas 방식
start = time.time()
df['result_pandas'] = np.sqrt(df['value1']**2 + df['value2']**2) + np.log(df['value3'] + 10)
pandas_time = time.time() - start

# NumPy 방식 (한 번에 배열로 변환)
start = time.time()
v1 = df['value1'].values
v2 = df['value2'].values
v3 = df['value3'].values
result = np.sqrt(v1**2 + v2**2) + np.log(v3 + 10)
df['result_numpy'] = result
numpy_time = time.time() - start

print(f"Pandas: {pandas_time:.4f}초")
print(f"NumPy:  {numpy_time:.4f}초")
print(f"속도 향상: {pandas_time/numpy_time:.1f}배")
print(f"결과 동일: {np.allclose(df['result_pandas'], df['result_numpy'])}")


2. 복잡한 수식 계산
----------------------------------------------------------------------
Pandas: 0.1465초
NumPy:  0.1074초
속도 향상: 1.4배
결과 동일: True


In [5]:
# 예제 3: 이동 평균 (Rolling) - 넘파이 승

print("\n" + "=" * 70)
print("3. 이동 평균 계산")
print("-" * 70)

# 샘플 데이터 
sample_df = df.head(10_000_000).copy()
window = 100

# Pandas rolling (상대적으로 느림)
start = time.time()
rolling_pandas = sample_df['score'].rolling(window=window).mean()
pandas_time = time.time() - start

# NumPy cumsum을 이용한 최적화
start = time.time()
scores = sample_df['score'].values
cumsum = np.cumsum(scores)
# cumsum[i] - cumsum[i-window] = sum of window
rolling_numpy = np.empty(len(scores))
rolling_numpy[:window-1] = np.nan
rolling_numpy[window-1:] = (cumsum[window-1:] - np.concatenate(([0], cumsum[:-window]))) / window
numpy_time = time.time() - start

print(f"Pandas rolling: {pandas_time:.4f}초")
print(f"NumPy cumsum:   {numpy_time:.4f}초")
print(f"속도 향상: {pandas_time/numpy_time:.1f}배")


3. 이동 평균 계산
----------------------------------------------------------------------
Pandas rolling: 0.1627초
NumPy cumsum:   0.0692초
속도 향상: 2.4배


In [6]:
# 예제 4: 정규화 (여러 컬럼 동시에) 판다스 승

print("\n" + "=" * 70)
print("4. 여러 컬럼 동시 정규화")
print("-" * 70)

numeric_cols = ['age', 'income', 'score', 'value1', 'value2', 'value3']

# Pandas 방식 (컬럼별 반복)
start = time.time()
df_normalized_pandas = df.copy()
for col in numeric_cols:
    min_val = df_normalized_pandas[col].min()
    max_val = df_normalized_pandas[col].max()
    df_normalized_pandas[col] = (df_normalized_pandas[col] - min_val) / (max_val - min_val)
pandas_time = time.time() - start

# NumPy 방식 (한 번에 처리)
start = time.time()
df_normalized_numpy = df.copy()
data_array = df_normalized_numpy[numeric_cols].values  # 2D NumPy 배열
min_vals = data_array.min(axis=0)
max_vals = data_array.max(axis=0)
normalized = (data_array - min_vals) / (max_vals - min_vals)
df_normalized_numpy[numeric_cols] = normalized
numpy_time = time.time() - start

print(f"Pandas (반복): {pandas_time:.4f}초")
print(f"NumPy (벡터화): {numpy_time:.4f}초")
print(f"속도 향상: {pandas_time/numpy_time:.1f}배")


4. 여러 컬럼 동시 정규화
----------------------------------------------------------------------
Pandas (반복): 1.3370초
NumPy (벡터화): 2.6184초
속도 향상: 0.5배


In [7]:
# 예제 5: 거리 계산 (유사도 매트릭스), 넘파이 승

print("\n" + "=" * 70)
print("5. 유클리디안 거리 행렬 (샘플 1000개)")
print("-" * 70)

# 작은 샘플로 테스트
sample_data = df[['value1', 'value2', 'value3']].head(1000).values

# Pandas 방식 (이중 반복문)
start = time.time()
n = len(sample_data)
distances_pandas = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        distances_pandas[i, j] = np.sqrt(((sample_data[i] - sample_data[j])**2).sum())
pandas_time = time.time() - start

# NumPy 브로드캐스팅
start = time.time()
diff = sample_data[:, np.newaxis, :] - sample_data[np.newaxis, :, :]  # (1000, 1000, 3)
distances_numpy = np.sqrt((diff**2).sum(axis=2))  # (1000, 1000)
numpy_time = time.time() - start

print(f"Pandas (이중 반복): {pandas_time:.4f}초")
print(f"NumPy (브로드캐스팅): {numpy_time:.4f}초")
print(f"속도 향상: {pandas_time/numpy_time:.1f}배")
print(f"결과 동일: {np.allclose(distances_pandas, distances_numpy)}")


5. 유클리디안 거리 행렬 (샘플 1000개)
----------------------------------------------------------------------
Pandas (이중 반복): 2.3443초
NumPy (브로드캐스팅): 0.0099초
속도 향상: 236.0배
결과 동일: True


In [ ]:
# 예제 6: 그룹별 통계 (커스텀 계산)은 비슷.

print("\n" + "=" * 70)
print("6. 그룹별 복잡한 통계 계산")
print("-" * 70)

# 예: 각 user_id별로 score의 표준편차 * income의 평균

# Pandas groupby 
start = time.time()
result_pandas = df.groupby('user_id').apply(
    lambda x: x['score'].std() * x['income'].mean()
).reset_index()
pandas_time = time.time() - start

# NumPy 방식
start = time.time()
# user_id로 정렬
df_sorted = df.sort_values('user_id')
user_ids = df_sorted['user_id'].values
scores = df_sorted['score'].values
incomes = df_sorted['income'].values

# 각 그룹의 시작/끝 인덱스 찾기
unique_users, group_starts = np.unique(user_ids, return_index=True)
group_ends = np.concatenate([group_starts[1:], [len(user_ids)]])

results = np.zeros(len(unique_users))
for i, (start_idx, end_idx) in enumerate(zip(group_starts, group_ends)):
    group_scores = scores[start_idx:end_idx]
    group_incomes = incomes[start_idx:end_idx]
    results[i] = np.std(group_scores) * np.mean(group_incomes)

result_numpy = pd.DataFrame({'user_id': unique_users, 0: results})
numpy_time = time.time() - start

print(f"Pandas groupby: {pandas_time:.4f}초")
print(f"NumPy:          {numpy_time:.4f}초")
print(f"속도 향상: {pandas_time/numpy_time:.1f}배")


6. 그룹별 복잡한 통계 계산
----------------------------------------------------------------------


/var/folders/d7/zpz29j7969dgtj_3mcj9r0sc0000gn/T/ipykernel_48753/2274060377.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_pandas = df.groupby('user_id').apply(


Pandas groupby: 2.4215초
NumPy:          2.3082초
속도 향상: 1.0배


#### Pandas가 내부적으로 NumPy 배열을 사용하는 예제

In [9]:
# dtype 최적화

print("\n" + "=" * 70)
print("보너스: 메모리 최적화")
print("=" * 70)

print(f"원본 메모리 사용량: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# dtype 최적화
df_optimized = df.copy()
df_optimized['user_id']  = df_optimized['user_id'].astype('int32')
df_optimized['age']      = df_optimized['age'].astype('int8')
df_optimized['income']   = df_optimized['income'].astype('int32')
df_optimized['score']    = df_optimized['score'].astype('float32')
df_optimized['category'] = df_optimized['category'].astype('category')

print(f"최적화 후 메모리: {df_optimized.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"절약: {(1 - df_optimized.memory_usage(deep=True).sum() / df.memory_usage(deep=True).sum()) * 100:.1f}%")


보너스: 메모리 최적화
원본 메모리 사용량: 1239.78 MB
최적화 후 메모리: 591.28 MB
절약: 52.3%
